<a href="https://colab.research.google.com/github/zh19980811/Code_Perference_Benchmark/blob/main/%E4%BB%A3%E7%A0%81%E9%A3%8E%E6%A0%BC%E9%A3%8E%E6%A0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ast
import re

# 🟢 工具函数
def is_snake_case(name):
    return bool(re.fullmatch(r'[a-z_][a-z0-9_]*', name))

def count_return_statements(node):
    return sum(isinstance(n, ast.Return) for n in ast.walk(node))

def has_try_except(node):
    return any(isinstance(n, ast.Try) for n in ast.walk(node))

def get_call_depth(node, depth=0):
    """递归计算最大嵌套调用深度"""
    if isinstance(node, ast.Call):
        return max([get_call_depth(arg, depth + 1) for arg in ast.iter_child_nodes(node)] + [depth])
    else:
        return max([get_call_depth(child, depth) for child in ast.iter_child_nodes(node)] + [depth])

# 🧠 方法打分主函数
def get_function_score(func_node, verbose=False):
    score = 0
    total = 7
    details = {}

    # 方法长度
    if hasattr(func_node, 'end_lineno'):
        length = func_node.end_lineno - func_node.lineno + 1
        length_score = 1 if length <= 20 else 0.5 if length <= 50 else 0
        score += length_score
        details["长度"] = (length, length_score)

    # 参数数量
    arg_count = len(func_node.args.args)
    arg_score = 1 if arg_count <= 3 else 0.5 if arg_count <= 5 else 0
    score += arg_score
    details["参数数"] = (arg_count, arg_score)

    # Docstring
    has_doc = ast.get_docstring(func_node) is not None
    doc_score = 1 if has_doc else 0
    score += doc_score
    details["docstring"] = (has_doc, doc_score)

    # Try/Except
    has_try = has_try_except(func_node)
    try_score = 1 if has_try else 0
    score += try_score
    details["异常处理"] = (has_try, try_score)

    # 命名风格
    name_ok = is_snake_case(func_node.name)
    name_score = 1 if name_ok else 0
    score += name_score
    details["命名规范"] = (func_node.name, name_score)

    # return 数量
    return_count = count_return_statements(func_node)
    ret_score = 1 if 1 <= return_count <= 3 else 0
    score += ret_score
    details["return数"] = (return_count, ret_score)

    # 调用嵌套深度
    max_depth = get_call_depth(func_node)
    depth_score = 1 if max_depth <= 3 else 0
    score += depth_score
    details["调用深度"] = (max_depth, depth_score)

    final_score = round((score / total) * 10, 2)

    if verbose:
        return final_score, details
    return final_score

# 🚀 主分析函数（输入代码字符串）
def analyze_code(code_str, verbose=True):
    tree = ast.parse(code_str)
    results = []
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            score, details = get_function_score(node, verbose=True)
            results.append((node.name, score, details))
    return results


In [2]:
code = '''
def fetch_data(url, retries=3):
    """Fetch data from the internet."""
    for _ in range(retries):
        try:
            response = request.get(url)
            return response.json()
        except:
            continue
    return None

def badFunctionName(a, b, c, d, e, f):
    try:
        return a + b + c + d + e + f
    except Exception as e:
        print(e)
'''

results = analyze_code(code)

# 显示结果
for name, score, detail in results:
    print(f"\n方法名: {name} → 得分: {score}/10")
    for key, (val, s) in detail.items():
        print(f"  {key}: {val} → +{s}")



方法名: fetch_data → 得分: 10.0/10
  长度: 9 → +1
  参数数: 2 → +1
  docstring: True → +1
  异常处理: True → +1
  命名规范: fetch_data → +1
  return数: 2 → +1
  调用深度: 1 → +1

方法名: badFunctionName → 得分: 5.71/10
  长度: 5 → +1
  参数数: 6 → +0
  docstring: False → +0
  异常处理: True → +1
  命名规范: badFunctionName → +0
  return数: 1 → +1
  调用深度: 1 → +1
